In [19]:
from typing import TypedDict, Optional

In [20]:
class TestDict(TypedDict):
    key1: str
    key2: int
    key3: bool
    optional: Optional[str] = None

In [21]:
test_dict = TestDict(key1="value1", key2=42, key3=True)

In [22]:
test_dict 

{'key1': 'value1', 'key2': 42, 'key3': True}

In [9]:
class MyClass:
    def __init__(self, test_dict: TestDict):
        self.test_dict = test_dict

    def get_value(self, key: str) -> str:
        return self.test_dict.get(key, "Key not found")
    
    def update_value(self, key: str, value: str) -> None:
        if key in self.test_dict:
            self.test_dict[key] = value
        else:
            raise KeyError(f"Key '{key}' not found in the dictionary")

In [3]:
test_dict = TestDict(key1="value1", key2=42, key3=True)

In [10]:
my_class = MyClass(test_dict)

In [17]:
my_class.test_dict.key1

AttributeError: 'dict' object has no attribute 'key1'

In [90]:
from pydantic import BaseModel
import json
import os, sys
from typing import Self, Union, Optional, Type, TypeVar
from abc import abstractmethod

T = TypeVar("T", bound="Config")

class Config(BaseModel):
    # TODO: Better name for this maybe?
    #       Make this class:
    #       - Be able to load itself from json and save itself to json
    #       - Be able to load itself from Self or str
    #       - Raise warnings/errors if paths to source jsons are not found
    #       - Raise warning (?) and make a path if destination json path does not exist
    #       - PS. Check if BaseModel does this already
    @classmethod
    def load(
        cls: Type[T],
        source: Optional[Union[str, T]] = None
    ) -> T:
        
        if isinstance(source, str):
            return cls.load_from_json(source)
        elif isinstance(source, cls):
            return source
        elif source is None:
            return cls.make_default()
        
    def save_to_json(
        self,
        config_file: str,
        **json_kwargs: dict
    ) -> None:
        
        with open(config_file, "w") as f:
            f.write(self.model_dump_json(**json_kwargs))

    @classmethod
    def load_from_json(
        cls: Type[T],
        path: str
    ) -> Self:
        
        try:  
            with open(path, "r") as f:
                loaded_class = cls.model_validate_json(f.read())
        except Exception as e:
            print(f"While reading {path}:{e}")
            sys.exit(1)
        
        return loaded_class
    
    @abstractmethod
    def make_default(cls: Type[T]) -> Self:
        ...

In [91]:
class TestConfig(Config):
    key1: str
    key2: int

    @classmethod
    def make_default(cls) -> Self:
        return cls(
            key1="default_value1",
            key2=42
        )

In [87]:
test_config = TestConfig.load(None)

In [94]:
test_config.save_to_json(path="configs/new_folder2/new_folder3/", name="test_config")

In [89]:
test_config = TestConfig.load("configs/test_configs.json")

While reading configs/test_configs.json:[Errno 2] No such file or directory: 'configs/test_configs.json'


SystemExit: 3

In [79]:
test_config

TestConfig(key1='default_value1', key2=11)